# Examine PIES data from pre-SWOT campaign

Jinbo Wang says there was not much internal tide signal in the PIES record

@jtomfarrar

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

################
# This allows us to import Tom_tools
import sys
sys.path.append('../Tom_tools/') # you may need to adjust this path
import Tom_tools_v1 as tt
################


In [68]:
# %matplotlib inline
%matplotlib qt5

plt.rcParams['figure.dpi'] = 100
plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams['savefig.dpi'] = 500
plt.close('all')

__figdir__ = "./plots/"
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [10]:
path = 'C:/Users/jtomf/Documents/Python/preSWOT_bottom_pressure/data/'
filename = 'SIO_PIES_L2_20190906_20200118_DM_VER003.nc'

In [11]:
path

'C:/Users/jtomf/Documents/Python/preSWOT_bottom_pressure/data/'

In [12]:
ds = xr.open_dataset(path+filename)

In [13]:
ds

<xarray.Dataset>
Dimensions:                  (DEPTH: 1, LATITUDE: 1, LONGITUDE: 1, STARTEND: 2, TIMEP: 19401, TIMET: 77604)
Coordinates:
  * LATITUDE                 (LATITUDE) float64 35.85
  * LONGITUDE                (LONGITUDE) float64 -125.0
  * DEPTH                    (DEPTH) float64 4.471e+03
  * TIMEP                    (TIMEP) datetime64[ns] 2019-09-06T00:11:22.00072...
  * TIMET                    (TIMET) datetime64[ns] 2019-09-06T00:10:00.00072...
Dimensions without coordinates: STARTEND
Data variables:
    TIMEP_BOUNDS             (STARTEND, TIMEP) datetime64[ns] ...
    SEAFLOOR_PRESSURE        (TIMEP) float64 ...
    SEAFLOOR_PRESSURE_QC     (TIMEP) int8 ...
    ACOUSTIC_TRAVEL_TIME     (TIMET) float64 ...
    ACOUSTIC_TRAVEL_TIME_QC  (TIMET) int8 ...
Attributes:
    Conventions:                   CF-1.7,ACDD-1.3
    featureType:                   timeSeries
    title:                         Data from SIO PIES Instrument During the S...
    summary:                       From September 2019 to January 2020, a num...
    source:                        seafloor observation
    institution:                   Scripps Institution of Oceanography
    publisher_name:                Matthias Lankhorst
    publisher_url:                 http://orcid.org/0000-0002-4166-4044
    contributor_name:              Jeff Sevadjian
    contributor_role:              Data Manager
    principal_investigator:        Uwe Send, Andrew Lucas, Robert Pinkel
    processing_level:              Instrument data converted to geophysical v...
    acknowledgement:               This study was supported by NASA's SWOT pr...
    geospatial_lat_min:            35.84583333333333
    geospatial_lat_max:            35.84583333333333
    geospatial_lon_min:            -125.04916666666666
    geospatial_lon_max:            -125.04916666666666
    geospatial_vertical_min:       4471.256064914745
    geospatial_vertical_max:       4471.256064914745
    geospatial_vertical_positive:  down
    geospatial_vertical_units:     m
    time_coverage_start:           20190906T001122Z
    time_coverage_end:             20200118T173136Z

In [35]:
fig = plt.figure(figsize=(12,5))
ax = fig.subplots(2, 1, sharex=True)
plt.axes(ax[0])
plt.plot(ds['TIMET'],ds['ACOUSTIC_TRAVEL_TIME'])
plt.legend('Acoustic travel time')
plt.ylabel('$\tau$ [s]')
plt.axes(ax[1])
plt.plot(ds['TIMET'],ds['ACOUSTIC_TRAVEL_TIME_QC'],'o')


# fig.subplots(sharex=True)

In [53]:
# Set up start time, stop time, and delta_t for bin averaging travel time data
t1 = np.datetime64(ds.TIMEP.dt.strftime("%Y-%m-%d %H:"+'00').values[0])
t2 = np.datetime64(ds.TIMEP.dt.strftime("%Y-%m-%d %H:"+'00').values[-1])
delta_t = np.timedelta64(2, 'h')

What I am trying to do is to bi average the travel time data.  A few minutes google searching turned up these possibilities.
https://stackoverflow.com/questions/6163334/binning-data-in-python-with-scipy-numpy

I want to bin-average the data to hourly:
consider:
```
bin_means = (numpy.histogram(data, bins, weights=data)[0] /
             numpy.histogram(data, bins)[0])
```
or 
```
from scipy.stats import binned_statistic
data = np.random.rand(100)
bin_means = binned_statistic(data, data, bins=10, range=(0, 1))[0]
```


Right now, I am basically stuck on the histogram approach (probably just not doing it right).

In [76]:
np.flatnonzero(ds['ACOUSTIC_TRAVEL_TIME_QC']==0)

array([   45,    46,    47, ..., 77589, 77590, 77591], dtype=int64)

In [79]:
bins = np.arange(t1,t2,delta_t)
data = ds['ACOUSTIC_TRAVEL_TIME'].isel(ds['ACOUSTIC_TRAVEL_TIME_QC']==0)
# bin_means = (np.histogram(ds['TIMET'].where(ds['ACOUSTIC_TRAVEL_TIME_QC']==0), bins, data)[0] /
#              np.histogram(ds['TIMET'].where(ds['ACOUSTIC_TRAVEL_TIME_QC']==0), bins)[0])
bin_means = np.histogram(ds['TIMET'].isel(np.flatnonzero(ds['ACOUSTIC_TRAVEL_TIME_QC']==0)), bins, data)[0]

# bin_means = np.histogram(ds['TIMET'].where(ds['ACOUSTIC_TRAVEL_TIME_QC']==0), bins)[0] 

ValueError: the first argument to .isel must be a dictionary

In [81]:
from scipy.stats import binned_statistic

bins = np.arange(t1,t2,delta_t)
data = ds['ACOUSTIC_TRAVEL_TIME'].where(ds['ACOUSTIC_TRAVEL_TIME_QC']==0)

bin_means = binned_statistic(ds['TIMET'].where(ds['ACOUSTIC_TRAVEL_TIME_QC']==0), data, statistic='median', bins=bins)[0]


TypeError: invalid type promotion

In [71]:
fig = plt.figure(figsize=(12,5))
plt.plot(bin_means)



In [45]:
# I want to bin-average the data to hourly:
# consider:


bin_means = (numpy.histogram(data, bins, weights=data)[0] /
             numpy.histogram(data, bins)[0])

# or 
from scipy.stats import binned_statistic

data = np.random.rand(100)
bin_means = binned_statistic(data, data, bins=10, range=(0, 1))[0]

NameError: name 'numpy' is not defined